In [1]:
%autosave 10
import time

Autosaving every 10 seconds


In [2]:
import openpyxl
import pandas as pd

# Please ensure that the Length of the Final Cuts are rounded off considerably

In case of mismatch due to algorithm rounding off after the Pattern Generation sheet is obtained, it may be manually matched

In [3]:
DashBoard_Obj=pd.read_excel("DashBoard.xlsx","Objective",index_col=0)
print(DashBoard_Obj)

                                                    Value  \
Parameter                                                   
Problem                                                20   
Objective                                               4   
Number of Combinations to be Generated                 30   
Timelimit for each of the smallest problems            30   
Timelimit for the Large CSP problems                  500   
Maximum Number of Unique Standard Length usages         2   
Maximum Number of Unique Pattern usages per Sta...     50   

                                                                                              Remarks  
Parameter                                                                                              
Problem                                                                  Diameter of the Bars (in mm)  
Objective                                           Case Type 4 = Generating the Top Minimum Waste...  
Number of Combinations to be Gener

In [4]:
# Create a dictionary from the index and a specific column
Decisions = DashBoard_Obj['Value'].to_dict()
print(Decisions)
# Decisions["Maximum Number of (Unique) Standard Length usages"]

{'Problem': 20, 'Objective': 4, 'Number of Combinations to be Generated': 30, 'Timelimit for each of the smallest problems': 30, 'Timelimit for the Large CSP problems': 500, 'Maximum Number of Unique Standard Length usages': 2, 'Maximum Number of Unique Pattern usages per Standard Length': 50}


In [5]:
DashBoard_Dataset_Cuts=pd.read_excel("DashBoard.xlsx","Dataset_Cuts",index_col=0)
print(DashBoard_Dataset_Cuts)

                         Total Requirements
Lengths of Cuts Allowed                    
3.951                                    48
3.953                                    48
3.960                                    48
4.046                                    24
4.230                                   332
4.331                                    24
4.332                                     8
4.337                                    32
4.344                                    24
4.345                                    24
4.364                                    29
4.478                                     9
4.503                                    24
4.549                                    24
4.730                                  1188
5.755                                    14
6.455                                    32
6.458                                    16
6.492                                    16
6.497                                    32
6.569                           

In [6]:
DashBoard_Dataset_StandardLengths=pd.read_excel("DashBoard.xlsx","Dataset_StandardLengths",index_col=0)
#print(type(DashBoard_Dataset_StandardLengths))
print(DashBoard_Dataset_StandardLengths)

                                Associated Cost
Total Standard Lengths Allowed                 
8.5                                    0.267035
9.0                                    0.282743
9.5                                    0.298451
10.0                                   0.314159
10.5                                   0.329867


In [7]:
length_of_cuts_allowed = DashBoard_Dataset_Cuts.index.tolist()
print(length_of_cuts_allowed)

[3.951, 3.953, 3.96, 4.046, 4.23, 4.331, 4.332, 4.337, 4.344, 4.345, 4.364, 4.478, 4.503, 4.549, 4.73, 5.755, 6.455, 6.458, 6.492, 6.497, 6.569, 6.573, 6.586, 6.757]


In [8]:
requirements = DashBoard_Dataset_Cuts["Total Requirements"]
print(requirements)
#requirements[1.2]

Lengths of Cuts Allowed
3.951      48
3.953      48
3.960      48
4.046      24
4.230     332
4.331      24
4.332       8
4.337      32
4.344      24
4.345      24
4.364      29
4.478       9
4.503      24
4.549      24
4.730    1188
5.755      14
6.455      32
6.458      16
6.492      16
6.497      32
6.569      32
6.573      32
6.586      32
6.757      16
Name: Total Requirements, dtype: int64


In [9]:
total_max_standardlengths_allowed = DashBoard_Dataset_StandardLengths.index.tolist()
print(total_max_standardlengths_allowed)

[8.5, 9.0, 9.5, 10.0, 10.5]


In [10]:
combinations = range(1,int(Decisions["Number of Combinations to be Generated"])+1)
print(combinations)

range(1, 31)


In [11]:
#max_time_in_seconds = Decisions["Timelimit for each of the smallest problems"]

#Timelimit_for_each_final_CSP_problem_minimizing_WCSP = Decisions["Timelimit for the Large CSP problems"]

# Generating Combinations Depending on Objectives

In [12]:
import gurobipy as gp
from gurobipy import GRB

In [13]:
start_time_is_this=time.time()
print(start_time_is_this)

1702800922.5391674


# Constraints dependant on Objectives

In [14]:
if Decisions["Objective"] == 4:
    M = 10000000
    X_Standards = {}
    
    for max_length in total_max_standardlengths_allowed:
        print("Max Standard Length Allowed: ", max_length)
        X_store = {}
        
        for c in combinations:
            # Create a new Gurobi model
            prob = gp.Model("Stock Cutting with multiple Master Lengths for Minimum Waste")

            # Variables
            x = {}
            m = {}
            g = {}

            for l in length_of_cuts_allowed:
                x[l] = prob.addVar(lb=0, vtype=GRB.INTEGER, name=f'x[{l}]')

            # Objective: Minimize Waste
            prob.setObjective(max_length - gp.quicksum(x[l] * l for l in length_of_cuts_allowed), sense=GRB.MINIMIZE)

            # Constraint 1
            prob.addConstr(gp.quicksum(x[l] * l for l in length_of_cuts_allowed) <= max_length)

            # Constraint 2
            for cc in combinations:
                if cc < c:
                    for l in length_of_cuts_allowed:
                        m[cc, l] = prob.addVar(lb=0, name=f'm[{cc},{l}]')
                        g[cc, l] = prob.addVar(vtype=GRB.BINARY, name=f'g[{cc},{l}]')

                        prob.addConstr(m[cc, l] >= X_store[cc, l] - x[l])
                        prob.addConstr(m[cc, l] >= x[l] - X_store[cc, l])

                        prob.addConstr(x[l] - X_store[cc, l] <= M * (1 - g[cc, l]))
                        prob.addConstr(X_store[cc, l] - x[l] <= M * g[cc, l])

                        prob.addConstr(m[cc, l] <= X_store[cc, l] - x[l] + M * (1 - g[cc, l]))
                        prob.addConstr(m[cc, l] <= x[l] - X_store[cc, l] + M * g[cc, l])

                    prob.addConstr(gp.quicksum(m[cc, l] for l in length_of_cuts_allowed) >= 1)

            # Solve the problem
            #prob.Params.TimeLimit = max_time_in_seconds
            prob.optimize()
            waste_generated = prob.objVal

            for l in length_of_cuts_allowed:
                X_store[c, l] = x[l].x
                if X_store[c, l] > 0:
                    print("\t X_store[", c, ",", l, "] = ", X_store[c, l])

            print("\t \t Total Waste Generated: ", waste_generated)
            X_store[c, "Waste Objective"] = waste_generated
            print("\t \t Total Time Taken: ", round(prob.Runtime, 1), " seconds, for this smallest problem of Standard length: ", max_length)

        X_Standards[max_length] = X_store.copy()

Max Standard Length Allowed:  8.5
Set parameter Username
Academic license - for non-commercial use only - expires 2024-12-10
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (win64)

CPU model: Intel(R) Core(TM) i3-10110U CPU @ 2.10GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 1 rows, 24 columns and 24 nonzeros
Model fingerprint: 0x36f9d956
Variable types: 0 continuous, 24 integer (0 binary)
Coefficient statistics:
  Matrix range     [4e+00, 7e+00]
  Objective range  [4e+00, 7e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [9e+00, 9e+00]
Found heuristic solution: objective 0.5980000
Presolve removed 0 rows and 10 columns
Presolve time: 0.00s
Presolved: 1 rows, 14 columns, 14 nonzeros
Variable types: 0 continuous, 14 integer (9 binary)
Found heuristic solution: objective 0.0000000

Root relaxation: cutoff, 0 iterations, 0.00 seconds (0.00 work units)

Explored 1 nodes (0 simplex iteratio

Model fingerprint: 0xf0457e6c
Variable types: 96 continuous, 120 integer (96 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+07]
  Objective range  [4e+00, 7e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+07]
Presolve removed 558 rows and 196 columns
Presolve time: 0.01s
Presolved: 23 rows, 20 columns, 88 nonzeros
Variable types: 0 continuous, 20 integer (15 binary)
Found heuristic solution: objective 0.0620000

Root relaxation: objective 8.881784e-16, 8 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    0.00000    0    2    0.06200    0.00000   100%     -    0s
H    0     0                       0.0460000    0.00000   100%     -    0s
     0     0    0.00000    0    3    0.04600    0.00000   100%     -    0s
     0     0    0.00000    0    2    0.04600    0.00000   100%     -    0s
     0    

     0     0    0.00000    0    3    0.09000    0.00000   100%     -    0s
     0     0    0.00000    0    3    0.09000    0.00000   100%     -    0s
     0     0    0.00000    0    5    0.09000    0.00000   100%     -    0s
     0     0    0.00000    0    3    0.09000    0.00000   100%     -    0s
     0     0    0.00000    0    5    0.09000    0.00000   100%     -    0s
     0     0    0.00000    0    5    0.09000    0.00000   100%     -    0s
     0     0    0.00000    0    2    0.09000    0.00000   100%     -    0s
     0     0    0.00000    0    2    0.09000    0.00000   100%     -    0s
     0     2    0.00000    0    2    0.09000    0.00000   100%     -    0s
*   17     0               7       0.0710000    0.07100  0.00%   3.0    0s

Cutting planes:
  Gomory: 2
  Implied bound: 1
  Clique: 1
  MIR: 3
  StrongCG: 1
  Flow cover: 2
  Relax-and-lift: 1

Explored 18 nodes (81 simplex iterations) in 0.07 seconds (0.01 work units)
Thread count was 4 (of 4 available processors)

Soluti

Coefficient statistics:
  Matrix range     [1e+00, 1e+07]
  Objective range  [4e+00, 7e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+07]
Presolve removed 1386 rows and 472 columns
Presolve time: 0.01s
Presolved: 65 rows, 32 columns, 214 nonzeros
Variable types: 0 continuous, 32 integer (27 binary)
Found heuristic solution: objective 0.1850000

Root relaxation: objective 0.000000e+00, 24 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    0.00000    0    1    0.18500    0.00000   100%     -    0s
H    0     0                       0.1100000    0.00000   100%     -    0s
     0     0    0.00000    0    4    0.11000    0.00000   100%     -    0s
     0     0    0.00000    0    7    0.11000    0.00000   100%     -    0s
     0     0    0.00000    0    1    0.11000    0.00000   100%     -    0s
     0     0    0


Cutting planes:
  Learned: 6
  Cover: 3
  Implied bound: 2
  MIR: 5
  StrongCG: 1
  Mod-K: 3
  RLT: 2

Explored 23 nodes (232 simplex iterations) in 0.10 seconds (0.02 work units)
Thread count was 4 (of 4 available processors)

Solution count 2: 0.122 0.185 

Optimal solution found (tolerance 1.00e-04)
Best objective 1.220000000000e-01, best bound 1.220000000000e-01, gap 0.0000%
	 X_store[ 13 , 4.046 ] =  1.0
	 X_store[ 13 , 4.332 ] =  1.0
	 	 Total Waste Generated:  0.12199999999999989
	 	 Total Time Taken:  0.1  seconds, for this smallest problem of Standard length:  8.5
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (win64)

CPU model: Intel(R) Core(TM) i3-10110U CPU @ 2.10GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 1886 rows, 648 columns and 4704 nonzeros
Model fingerprint: 0x30399f1b
Variable types: 312 continuous, 336 integer (312 binary)
Coefficient statistics:
  Matrix range     [1e+0

...


    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    0.00000    0    2    1.81000    0.00000   100%     -    0s
H    0     0                       0.7940000    0.00000   100%     -    0s
     0     0    0.00000    0    7    0.79400    0.00000   100%     -    0s
     0     0    0.00000    0    2    0.79400    0.00000   100%     -    0s
     0     0    0.00000    0    3    0.79400    0.00000   100%     -    0s
H    0     0                       0.3810000    0.00000   100%     -    0s
     0     0    0.00000    0    1    0.38100    0.00000   100%     -    0s
     0     0    0.00000    0    5    0.38100    0.00000   100%     -    0s
     0     0    0.00000    0    7    0.38100    0.00000   100%     -    0s
     0     0    0.00000    0    5    0.38100    0.00000   100%     -    0s
     0     0    0.00000    0    9    0.38100    0.00000   100%     -    0s
     0     0    0.01900


    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    0.00000    0    1    2.00000    0.00000   100%     -    0s
H    0     0                       1.8120000    0.00000   100%     -    0s
     0     0    0.00000    0   10    1.81200    0.00000   100%     -    0s
H    0     0                       1.8110000    0.00000   100%     -    0s
     0     0    0.00000    0   13    1.81100    0.00000   100%     -    0s
     0     0    0.00000    0    2    1.81100    0.00000   100%     -    0s
H    0     0                       1.4260000    0.00000   100%     -    0s
H    0     0                       0.4080000    0.00000   100%     -    0s
     0     0    0.00000    0    3    0.40800    0.00000   100%     -    0s
     0     0    0.00000    0    3    0.40800    0.00000   100%     -    0s
     0     0    0.00000    0    5    0.40800    0.00000   100%     -    0s
     0     0    0.00000

     0     0    0.00000    0    8    0.51500    0.00000   100%     -    0s
     0     0    0.00000    0    9    0.51500    0.00000   100%     -    0s
     0     0    0.00000    0    5    0.51500    0.00000   100%     -    0s
     0     0    0.00000    0    9    0.51500    0.00000   100%     -    0s
     0     0    0.00000    0    9    0.51500    0.00000   100%     -    0s
     0     2    0.00000    0    9    0.51500    0.00000   100%     -    0s

Cutting planes:
  Learned: 3
  Cover: 1
  MIR: 8
  StrongCG: 7

Explored 211 nodes (600 simplex iterations) in 0.16 seconds (0.07 work units)
Thread count was 4 (of 4 available processors)

Solution count 3: 0.515 0.792 2.215 

Optimal solution found (tolerance 1.00e-04)
Best objective 5.150000000000e-01, best bound 5.150000000000e-01, gap 0.0000%
	 X_store[ 23 , 4.23 ] =  1.0
	 X_store[ 23 , 5.755 ] =  1.0
	 	 Total Waste Generated:  0.5150000000000006
	 	 Total Time Taken:  0.2  seconds, for this smallest problem of Standard length:  10.5
Gu


Optimize a model with 3626 rows, 1224 columns and 9024 nonzeros
Model fingerprint: 0xbd0939f5
Variable types: 600 continuous, 624 integer (600 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+07]
  Objective range  [4e+00, 7e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+07]
Presolve removed 3450 rows and 1153 columns
Presolve time: 0.04s
Presolved: 176 rows, 71 columns, 846 nonzeros
Variable types: 0 continuous, 71 integer (56 binary)
Found heuristic solution: objective 2.2080000

Root relaxation: objective 0.000000e+00, 60 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    0.00000    0    1    2.20800    0.00000   100%     -    0s
H    0     0                       1.8170000    0.00000   100%     -    0s
     0     0    0.00000    0    6    1.81700    0.00000   100%     -    0s
H    0     0 

Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (win64)

CPU model: Intel(R) Core(TM) i3-10110U CPU @ 2.10GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 4061 rows, 1368 columns and 10104 nonzeros
Model fingerprint: 0x873ffc4c
Variable types: 672 continuous, 696 integer (672 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+07]
  Objective range  [4e+00, 7e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+07]
Presolve removed 3870 rows and 1293 columns
Presolve time: 0.03s
Presolved: 191 rows, 75 columns, 933 nonzeros
Variable types: 0 continuous, 75 integer (60 binary)
Found heuristic solution: objective 1.8170000

Root relaxation: objective 0.000000e+00, 53 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    0

In [15]:
max_combination_for_each_standard = {}

if Decisions["Objective"] == 4:
    for max_length in total_max_standardlengths_allowed:
        print("Max Standard Length Allowed: ", max_length)
        X_store = {}
        combination_previous_end_number = Decisions["Number of Combinations to be Generated"]
        current_combination_number = combination_previous_end_number

        for ll in length_of_cuts_allowed:
            if ll > max_length:
                continue

            current_combination_number += 1

            # Create a new Gurobi model
            prob = gp.Model("Stock Cutting with multiple Master Lengths for Minimum Waste")

            # Variables
            x = {}
            m = {}
            g = {}

            for l in length_of_cuts_allowed:
                x[l] = prob.addVar(lb=0, vtype=GRB.INTEGER, name=f'x[{l}]')

            # Objective: Minimize Waste
            prob.setObjective(max_length - gp.quicksum(x[l] * l for l in length_of_cuts_allowed), sense=GRB.MINIMIZE)

            # Constraint 0
            prob.addConstr(x[ll] >= 1)

            # Constraint 1
            prob.addConstr(gp.quicksum(x[l] * l for l in length_of_cuts_allowed) <= max_length)

            # Constraint 2
            for cc in combinations:
                for l in length_of_cuts_allowed:
                    m[cc, l] = prob.addVar(lb=0, name=f'm[{cc},{l}]')
                    g[cc, l] = prob.addVar(vtype=GRB.BINARY, name=f'g[{cc},{l}]')

                    prob.addConstr(m[cc, l] >= X_Standards[max_length][cc, l] - x[l])
                    prob.addConstr(m[cc, l] >= x[l] - X_Standards[max_length][cc, l])

                    prob.addConstr(x[l] - X_Standards[max_length][cc, l] <= M * (1 - g[cc, l]))
                    prob.addConstr(X_Standards[max_length][cc, l] - x[l] <= M * g[cc, l])

                    prob.addConstr(m[cc, l] <= X_Standards[max_length][cc, l] - x[l] + M * (1 - g[cc, l]))
                    prob.addConstr(m[cc, l] <= x[l] - X_Standards[max_length][cc, l] + M * g[cc, l])

                prob.addConstr(gp.quicksum(m[cc, l] for l in length_of_cuts_allowed) >= 1)

            for cc in range(combination_previous_end_number + 1, current_combination_number):
                for l in length_of_cuts_allowed:
                    m[cc, l] = prob.addVar(lb=0, name=f'm[{cc},{l}]')
                    g[cc, l] = prob.addVar(vtype=GRB.BINARY, name=f'g[{cc},{l}]')

                    prob.addConstr(m[cc, l] >= X_store[cc, l] - x[l])
                    prob.addConstr(m[cc, l] >= x[l] - X_store[cc, l])

                    prob.addConstr(x[l] - X_store[cc, l] <= M * (1 - g[cc, l]))
                    prob.addConstr(X_store[cc, l] - x[l] <= M * g[cc, l])

                    prob.addConstr(m[cc, l] <= X_store[cc, l] - x[l] + M * (1 - g[cc, l]))
                    prob.addConstr(m[cc, l] <= x[l] - X_store[cc, l] + M * g[cc, l])

                prob.addConstr(gp.quicksum(m[cc, l] for l in length_of_cuts_allowed) >= 1)

            # Solve the problem
            #prob.Params.TimeLimit = max_time_in_seconds
            prob.optimize()
            
            
            
            if prob.SolCount:
                waste_generated = prob.objVal

                for l in length_of_cuts_allowed:
                    X_store[current_combination_number, l] = x[l].x
                    if X_store[current_combination_number, l] > 0:
                        print("\t X_store[", current_combination_number, ",", l, "] = ", X_store[current_combination_number, l])

                print("\t \t Total Waste Generated: ", waste_generated,
                      " during mandatory unique usage of cut-length: ", ll, "in standard length: ", max_length)
                X_store[current_combination_number, "Waste Objective"] = waste_generated
                print("\t \t Total Time Taken: ", round(prob.Runtime, 1), " seconds, for this smallest problem")
            else:
                current_combination_number-=1

        max_combination_for_each_standard[max_length] = current_combination_number
        X_Standards[max_length].update(X_store)

Max Standard Length Allowed:  8.5
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (win64)

CPU model: Intel(R) Core(TM) i3-10110U CPU @ 2.10GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 4352 rows, 1464 columns and 10825 nonzeros
Model fingerprint: 0xed49b1de
Variable types: 720 continuous, 744 integer (720 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+07]
  Objective range  [4e+00, 7e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+07]
Presolve removed 4352 rows and 1464 columns
Presolve time: 0.02s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.03 seconds (0.01 work units)
Thread count was 1 (of 4 available processors)

Solution count 1: 0.217 

Optimal solution found (tolerance 1.00e-04)
Best objective 2.170000000000e-01, best bound 2.170000000000e-01, gap 0.0000%
	 X_store[ 31 , 3.951 ] =  1.0
	 X_store[ 31 , 4.332 ] =  

Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 5367 rows, 1800 columns and 13345 nonzeros
Model fingerprint: 0xd45e0cff
Variable types: 888 continuous, 912 integer (888 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+07]
  Objective range  [4e+00, 7e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+07]
Presolve removed 5367 rows and 1800 columns
Presolve time: 0.01s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.03 seconds (0.01 work units)
Thread count was 1 (of 4 available processors)

Solution count 1: 4.163 

Optimal solution found (tolerance 1.00e-04)
Best objective 4.163000000000e+00, best bound 4.163000000000e+00, gap 0.0000%
	 X_store[ 38 , 4.337 ] =  1.0
	 	 Total Waste Generated:  4.163  during mandatory unique usage of cut-length:  4.337 in standard length:  8.5
	 	 Total Time Taken:  0.0  seconds, for this smallest problem
Gurobi Optimizer version 10.0.

Presolve removed 6382 rows and 2136 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.02 seconds (0.01 work units)
Thread count was 1 (of 4 available processors)

Solution count 1: 3.77 

Optimal solution found (tolerance 1.00e-04)
Best objective 3.770000000000e+00, best bound 3.770000000000e+00, gap 0.0000%
	 X_store[ 45 , 4.73 ] =  1.0
	 	 Total Waste Generated:  3.7699999999999996  during mandatory unique usage of cut-length:  4.73 in standard length:  8.5
	 	 Total Time Taken:  0.0  seconds, for this smallest problem
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (win64)

CPU model: Intel(R) Core(TM) i3-10110U CPU @ 2.10GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 6527 rows, 2184 columns and 16225 nonzeros
Model fingerprint: 0xf5629d99
Variable types: 1080 continuous, 1104 integer (1080 binary)
Coefficient statistics:
  Matrix 

	 X_store[ 52 , 6.573 ] =  1.0
	 	 Total Waste Generated:  1.9269999999999996  during mandatory unique usage of cut-length:  6.573 in standard length:  8.5
	 	 Total Time Taken:  0.0  seconds, for this smallest problem
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (win64)

CPU model: Intel(R) Core(TM) i3-10110U CPU @ 2.10GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 7542 rows, 2520 columns and 18745 nonzeros
Model fingerprint: 0xb802b56f
Variable types: 1248 continuous, 1272 integer (1248 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+07]
  Objective range  [4e+00, 7e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+07]
Presolve removed 7542 rows and 2520 columns
Presolve time: 0.01s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.03 seconds (0.01 work units)
Thread count was 1 (of 4 available processors)

Solution count 1: 

Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (win64)

CPU model: Intel(R) Core(TM) i3-10110U CPU @ 2.10GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 5077 rows, 1704 columns and 12625 nonzeros
Model fingerprint: 0xe6063463
Variable types: 840 continuous, 864 integer (840 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+07]
  Objective range  [4e+00, 7e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+07]
Presolve removed 5077 rows and 1704 columns
Presolve time: 0.01s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.02 seconds (0.01 work units)
Thread count was 1 (of 4 available processors)

Solution count 1: 0.305 

Optimal solution found (tolerance 1.00e-04)
Best objective 3.050000000000e-01, best bound 3.050000000000e-01, gap 0.0000%
	 X_store[ 36 , 4.331 ] =  1.0
	 X_store[ 36 , 4.364 ] =  1.0
	 	 Total Waste Generated:  0.

...

  RHS range        [1e+00, 1e+07]
Presolve removed 6382 rows and 2136 columns
Presolve time: 0.02s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.04 seconds (0.02 work units)
Thread count was 1 (of 4 available processors)

Solution count 1: 5.27 

Optimal solution found (tolerance 1.00e-04)
Best objective 5.270000000000e+00, best bound 5.270000000000e+00, gap 0.0000%
	 X_store[ 45 , 4.73 ] =  1.0
	 	 Total Waste Generated:  5.27  during mandatory unique usage of cut-length:  4.73 in standard length:  10.0
	 	 Total Time Taken:  0.0  seconds, for this smallest problem
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (win64)

CPU model: Intel(R) Core(TM) i3-10110U CPU @ 2.10GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 6527 rows, 2184 columns and 16225 nonzeros
Model fingerprint: 0x25a41a14
Variable types: 1080 continuous, 1104 integer (1080 binary)
Coefficient 

	 X_store[ 52 , 6.573 ] =  1.0
	 	 Total Waste Generated:  3.4269999999999996  during mandatory unique usage of cut-length:  6.573 in standard length:  10.0
	 	 Total Time Taken:  0.0  seconds, for this smallest problem
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (win64)

CPU model: Intel(R) Core(TM) i3-10110U CPU @ 2.10GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 7542 rows, 2520 columns and 18745 nonzeros
Model fingerprint: 0x8fd4a633
Variable types: 1248 continuous, 1272 integer (1248 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+07]
  Objective range  [4e+00, 7e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+07]
Presolve removed 7542 rows and 2520 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.02 seconds (0.01 work units)
Thread count was 1 (of 4 available processors)

Solution count 1:

Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (win64)

CPU model: Intel(R) Core(TM) i3-10110U CPU @ 2.10GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 5077 rows, 1704 columns and 12625 nonzeros
Model fingerprint: 0x751c646d
Variable types: 840 continuous, 864 integer (840 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+07]
  Objective range  [4e+00, 7e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+07]
Presolve removed 5077 rows and 1704 columns
Presolve time: 0.01s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.02 seconds (0.01 work units)
Thread count was 1 (of 4 available processors)

Solution count 1: 1.439 

Optimal solution found (tolerance 1.00e-04)
Best objective 1.439000000000e+00, best bound 1.439000000000e+00, gap 0.0000%
	 X_store[ 36 , 4.331 ] =  1.0
	 X_store[ 36 , 4.73 ] =  1.0
	 	 Total Waste Generated:  1.4

Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 6092 rows, 2040 columns and 15145 nonzeros
Model fingerprint: 0xfe759697
Variable types: 1008 continuous, 1032 integer (1008 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+07]
  Objective range  [4e+00, 7e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+07]
Presolve removed 6092 rows and 2040 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.02 seconds (0.01 work units)
Thread count was 1 (of 4 available processors)

Solution count 1: 1.448 

Optimal solution found (tolerance 1.00e-04)
Best objective 1.448000000000e+00, best bound 1.448000000000e+00, gap 0.0000%
	 X_store[ 43 , 4.503 ] =  1.0
	 X_store[ 43 , 4.549 ] =  1.0
	 	 Total Waste Generated:  1.4480000000000004  during mandatory unique usage of cut-length:  4.503 in standard length:  10.5
	 	 Total Time Taken:  0.0  seconds, for this

  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+07]
Presolve removed 7107 rows and 2376 columns
Presolve time: 0.01s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.01 work units)
Thread count was 1 (of 4 available processors)

Solution count 1: 4.003 

Optimal solution found (tolerance 1.00e-04)
Best objective 4.003000000000e+00, best bound 4.003000000000e+00, gap 0.0000%
	 X_store[ 50 , 6.497 ] =  1.0
	 	 Total Waste Generated:  4.003  during mandatory unique usage of cut-length:  6.497 in standard length:  10.5
	 	 Total Time Taken:  0.0  seconds, for this smallest problem
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (win64)

CPU model: Intel(R) Core(TM) i3-10110U CPU @ 2.10GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 7252 rows, 2424 columns and 18025 nonzeros
Model fingerprint: 0x6e50b604
Variable types: 1200 continuous, 1

In [16]:
import winsound
winsound.Beep(500,1000)
max_combination_for_each_standard

{8.5: 54, 9.0: 54, 9.5: 54, 10.0: 54, 10.5: 54}

In [17]:
wb=openpyxl.Workbook()

sheet_for_Combinations = wb.active

row_number=1
sheet_for_Combinations.cell(row=row_number,column=1).value= "Length of Cut"

column_number=2
for i,max_length in enumerate(total_max_standardlengths_allowed):
    #for c in range(1,current_combination_number+1):
    for c in range(1,max_combination_for_each_standard[max_length]+1):
        sheet_for_Combinations.cell(row=row_number,column= column_number).value="Combination "+str(column_number-1)
        column_number+=1




for l in length_of_cuts_allowed:
    row_number+=1
    sheet_for_Combinations.cell(row=row_number,column=1).value= l
    column_number=2
    for i,max_length in enumerate(total_max_standardlengths_allowed):
        #for c in range(1,current_combination_number+1):
        for c in range(1,max_combination_for_each_standard[max_length]+1):
            sheet_for_Combinations.cell(row=row_number,column= column_number).value= X_Standards[max_length][c,l]
            column_number+=1
        
        
row_number+=1
sheet_for_Combinations.cell(row=row_number,column=1).value= "Standard Max Length"
column_number=2
for i,max_length in enumerate(total_max_standardlengths_allowed):
    #for c in range(1,current_combination_number+1):
    for c in range(1,max_combination_for_each_standard[max_length]+1):
        sheet_for_Combinations.cell(row=row_number,column= column_number).value= max_length
        column_number+=1
        

        
row_number+=1
sheet_for_Combinations.cell(row=row_number,column=1).value= "Wasted Length"
column_number=2
for i,max_length in enumerate(total_max_standardlengths_allowed):
    #for c in range(1,current_combination_number+1):
    for c in range(1,max_combination_for_each_standard[max_length]+1):
        sheet_for_Combinations.cell(row=row_number,column= column_number).value= X_Standards[max_length][c,"Waste Objective"]
        column_number+=1

In [18]:
#file_path = "./Problem_"+str(Decisions["Problem"])+"__CombinationGeneration_"+str(Decisions["Number of Combinations to be Generated"])+"__TimeLimit_"+str(max_time_in_seconds)+".xlsx"
file_path = "./Problem_"+str(Decisions["Problem"])+"__CombinationGeneration_"+str(Decisions["Number of Combinations to be Generated"])+"__TimeLimit_"+".xlsx"
wb.save(file_path)
print(file_path)

./Problem_20__CombinationGeneration_30__TimeLimit_.xlsx


In [19]:
X_Standards

{8.5: {(1, 3.951): 1.0,
  (1, 3.953): 0.0,
  (1, 3.96): 0.0,
  (1, 4.046): 0.0,
  (1, 4.23): 0.0,
  (1, 4.331): 0.0,
  (1, 4.332): 0.0,
  (1, 4.337): 0.0,
  (1, 4.344): 0.0,
  (1, 4.345): 0.0,
  (1, 4.364): 0.0,
  (1, 4.478): 0.0,
  (1, 4.503): 0.0,
  (1, 4.549): 1.0,
  (1, 4.73): -0.0,
  (1, 5.755): -0.0,
  (1, 6.455): -0.0,
  (1, 6.458): -0.0,
  (1, 6.492): -0.0,
  (1, 6.497): -0.0,
  (1, 6.569): -0.0,
  (1, 6.573): -0.0,
  (1, 6.586): -0.0,
  (1, 6.757): 0.0,
  (1, 'Waste Objective'): 0.0,
  (2, 3.951): -0.0,
  (2, 3.953): -0.0,
  (2, 3.96): 1.0,
  (2, 4.046): -0.0,
  (2, 4.23): 0.0,
  (2, 4.331): -0.0,
  (2, 4.332): -0.0,
  (2, 4.337): -0.0,
  (2, 4.344): -0.0,
  (2, 4.345): -0.0,
  (2, 4.364): 0.0,
  (2, 4.478): 0.0,
  (2, 4.503): 1.0,
  (2, 4.549): 0.0,
  (2, 4.73): 0.0,
  (2, 5.755): -0.0,
  (2, 6.455): -0.0,
  (2, 6.458): -0.0,
  (2, 6.492): -0.0,
  (2, 6.497): 0.0,
  (2, 6.569): 0.0,
  (2, 6.573): 0.0,
  (2, 6.586): -0.0,
  (2, 6.757): -0.0,
  (2, 'Waste Objective'): 0.0369999

In [20]:
for max_length in total_max_standardlengths_allowed:
    #print(max_length)
    print(X_Standards[max_length])

{(1, 3.951): 1.0, (1, 3.953): 0.0, (1, 3.96): 0.0, (1, 4.046): 0.0, (1, 4.23): 0.0, (1, 4.331): 0.0, (1, 4.332): 0.0, (1, 4.337): 0.0, (1, 4.344): 0.0, (1, 4.345): 0.0, (1, 4.364): 0.0, (1, 4.478): 0.0, (1, 4.503): 0.0, (1, 4.549): 1.0, (1, 4.73): -0.0, (1, 5.755): -0.0, (1, 6.455): -0.0, (1, 6.458): -0.0, (1, 6.492): -0.0, (1, 6.497): -0.0, (1, 6.569): -0.0, (1, 6.573): -0.0, (1, 6.586): -0.0, (1, 6.757): 0.0, (1, 'Waste Objective'): 0.0, (2, 3.951): -0.0, (2, 3.953): -0.0, (2, 3.96): 1.0, (2, 4.046): -0.0, (2, 4.23): 0.0, (2, 4.331): -0.0, (2, 4.332): -0.0, (2, 4.337): -0.0, (2, 4.344): -0.0, (2, 4.345): -0.0, (2, 4.364): 0.0, (2, 4.478): 0.0, (2, 4.503): 1.0, (2, 4.549): 0.0, (2, 4.73): 0.0, (2, 5.755): -0.0, (2, 6.455): -0.0, (2, 6.458): -0.0, (2, 6.492): -0.0, (2, 6.497): 0.0, (2, 6.569): 0.0, (2, 6.573): 0.0, (2, 6.586): -0.0, (2, 6.757): -0.0, (2, 'Waste Objective'): 0.036999999999999034, (3, 3.951): 0.0, (3, 3.953): 0.0, (3, 3.96): 0.0, (3, 4.046): 0.0, (3, 4.23): 2.0, (3, 4.33

# Main CSProblem starts here:~

(designed to also run separately from the above program, be cautious about the intermediate file name changes if done manually)

In [21]:
import gurobipy as gp
from gurobipy import GRB

from math import ceil

In [22]:
patterns_df = pd.read_excel(file_path, index_col=0)
#print(patterns_df)

In [23]:
# Get the last two rows into a new DataFrame
last_two_rows = patterns_df.tail(2).copy()  # tail(2) gets the last 2 rows

# Remove the last two rows from the original DataFrame
new_df = patterns_df.drop(patterns_df.tail(2).index)

In [24]:
# Find unique elements in the first row (index 0)
unique_elements_row = last_two_rows.iloc[0].unique()

print("Unique elements in the first row of last_two_rows:")
print(unique_elements_row)

Unique elements in the first row of last_two_rows:
[ 8.5  9.   9.5 10.  10.5]


In [25]:
all_length_of_cuts_allowed = new_df.index.tolist() # we could have used length_of_cuts_allowed, but here we take data from the Excel Sheet as we may want to change something for Later (in the Excel Sheet manually) which is however not recommended nor necessary
all_length_of_cuts_allowed

[3.951,
 3.953,
 3.96,
 4.046,
 4.23,
 4.331,
 4.332,
 4.337,
 4.344,
 4.345,
 4.364,
 4.478,
 4.503,
 4.549,
 4.73,
 5.755,
 6.455,
 6.458,
 6.492,
 6.497,
 6.569,
 6.573,
 6.586,
 6.757]

# Waste, Cost, Standard, Pattern: The WCSP sequence of Minimization (4 sequential problems

In [26]:
# Constraints
sl_key__value_c_mapping_dict={}
set_of_encountered_sl=set()
for c in patterns_df:
    sl = last_two_rows[c]["Standard Max Length"]
    if sl in set_of_encountered_sl:
        sl_key__value_c_mapping_dict[sl].add(c)
    else:
        sl_key__value_c_mapping_dict[sl]={c}
        set_of_encountered_sl.add(sl)
    
#print(sl_key__value_c_mapping_dict)

# WASTE MINIMIZATION

In [27]:
# Create a Gurobi model
CSProb = gp.Model("Stock Cutting with Waste Minimization")

# Variables
x = {}

for c in patterns_df:
    x[c] = CSProb.addVar(lb=0, vtype=GRB.INTEGER, name=f'x[{c}]')


# Objective: Minimize Waste
CSProb.setObjective(gp.quicksum(x[c] * last_two_rows[c]["Wasted Length"] for c in patterns_df), sense=GRB.MINIMIZE)

# Satisfying Requirements
for l in all_length_of_cuts_allowed:
    CSProb.addConstr(gp.quicksum(x[c] * new_df[c][l] for c in patterns_df) >= requirements[l])





# Set a time limit for optimization (e.g., 60 seconds)
#https://www.gurobi.com/documentation/10.0/refman/python_parameter_examples.html#PythonParameterExamples
currentTIMElimit = CSProb.Params.timeLimit
print("Default Time Limit: ",currentTIMElimit)
#CSProb.Params.timeLimit = Timelimit_for_each_final_CSP_problem_minimizing_WCSP


currentIntFeasTolerancelimit = CSProb.Params.IntFeasTol
print("Default Interger Tolerance: ",currentIntFeasTolerancelimit)
CSProb.Params.IntFeasTol = 1e-9


#FeasibilityTol
#https://www.gurobi.com/documentation/current/refman/feasibilitytol.html#parameter:FeasibilityTol
currentFeasibilityTolerancelimit = CSProb.Params.FeasibilityTol
print("Default Feasibility Tolerance: ",currentFeasibilityTolerancelimit)
CSProb.Params.FeasibilityTol = 1e-9


#OptimalityTol
#https://www.gurobi.com/documentation/current/refman/optimalitytol.html
currentOptimalityTolerancelimit = CSProb.Params.OptimalityTol
print("Default Optimality Tolerance: ",currentOptimalityTolerancelimit)
CSProb.Params.OptimalityTol = 1e-9


currentMIPGapLimit = CSProb.Params.MIPGap
print("Default MIP Gap Limit: ",currentMIPGapLimit)
#CSProb.setParam('MIPGap', 0)
CSProb.Params.MIPGap = 0
print("Assigned MIP Gap Limit: ",currentMIPGapLimit)

#https://www.gurobi.com/documentation/current/refman/mip_models.html#:~:text=The%20MIPFocus%20parameter%20allows%20you%20to%20modify%20your,good%20quality%20feasible%20solutions%2C%20you%20can%20select%20MIPFocus%3D1.
#https://www.gurobi.com/documentation/current/refman/mipfocus.html
#https://support.gurobi.com/hc/en-us/community/posts/8791754564369-How-does-MIPFocus-really-work-
currentMIPFocus = CSProb.Params.MIPFocus
print("The current MIPFocus is: ",currentMIPFocus)
CSProb.Params.MIPFocus=2


# Optimize the model
CSProb.optimize()

Default Time Limit:  inf
Default Interger Tolerance:  1e-05
Set parameter IntFeasTol to value 1e-09
Default Feasibility Tolerance:  1e-06
Set parameter FeasibilityTol to value 1e-09
Default Optimality Tolerance:  1e-06
Set parameter OptimalityTol to value 1e-09
Default MIP Gap Limit:  0.0001
Set parameter MIPGap to value 0
Assigned MIP Gap Limit:  0.0001
The current MIPFocus is:  0
Set parameter MIPFocus to value 2
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (win64)

CPU model: Intel(R) Core(TM) i3-10110U CPU @ 2.10GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 24 rows, 270 columns and 467 nonzeros
Model fingerprint: 0x97906a68
Variable types: 0 continuous, 270 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+00]
  Objective range  [2e-02, 6e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [8e+00, 1e+03]
Found heuristic solution: objective 5244.8040000
Presolve

# COST MINIMIZATION

In [28]:
# Create a new Gurobi model
CSProb2 = gp.Model("Stock Cutting with Cost Minimization")

# Variables
x_2 = {}

for c in patterns_df:
    x_2[c] = CSProb2.addVar(lb=0, vtype=GRB.INTEGER, name=f'x_2[{c}]')



# Previous OBJECTIVE 1: Minimize Waste : Now treated as constraint
CSProb2.addConstr(gp.quicksum(x_2[c]*last_two_rows[c]["Wasted Length"] for c in patterns_df) <= ceil(CSProb.objVal *1000)/1000, name="Obj1 as Const.")


for l in all_length_of_cuts_allowed:
    CSProb2.addConstr(gp.quicksum(x_2[c] * new_df[c][l] for c in patterns_df) >= requirements[l])


# Objective 2
CSProb2.setObjective(gp.quicksum(x_2[c] * DashBoard_Dataset_StandardLengths["Associated Cost"][last_two_rows[c]["Standard Max Length"]] for c in patterns_df),
                     sense=GRB.MINIMIZE)






# Set a time limit for optimization (e.g., 60 seconds)
#https://www.gurobi.com/documentation/10.0/refman/python_parameter_examples.html#PythonParameterExamples
currentTIMElimit = CSProb2.Params.timeLimit
print("Default Time Limit: ",currentTIMElimit)
#CSProb2.Params.timeLimit = Timelimit_for_each_final_CSP_problem_minimizing_WCSP


currentIntFeasTolerancelimit = CSProb2.Params.IntFeasTol
print("Default Interger Tolerance: ",currentIntFeasTolerancelimit)
CSProb2.Params.IntFeasTol = 1e-9


#FeasibilityTol
#https://www.gurobi.com/documentation/current/refman/feasibilitytol.html#parameter:FeasibilityTol
currentFeasibilityTolerancelimit = CSProb2.Params.FeasibilityTol
print("Default Feasibility Tolerance: ",currentFeasibilityTolerancelimit)
CSProb2.Params.FeasibilityTol = 1e-9


#OptimalityTol
#https://www.gurobi.com/documentation/current/refman/optimalitytol.html
currentOptimalityTolerancelimit = CSProb2.Params.OptimalityTol
print("Default Optimality Tolerance: ",currentOptimalityTolerancelimit)
CSProb2.Params.OptimalityTol = 1e-9


currentMIPGapLimit = CSProb2.Params.MIPGap
print("Default MIP Gap Limit: ",currentMIPGapLimit)
#CSProb.setParam('MIPGap', 0)
CSProb2.Params.MIPGap = 0
print("Assigned MIP Gap Limit: ",currentMIPGapLimit)

#https://www.gurobi.com/documentation/current/refman/mip_models.html#:~:text=The%20MIPFocus%20parameter%20allows%20you%20to%20modify%20your,good%20quality%20feasible%20solutions%2C%20you%20can%20select%20MIPFocus%3D1.
#https://www.gurobi.com/documentation/current/refman/mipfocus.html
#https://support.gurobi.com/hc/en-us/community/posts/8791754564369-How-does-MIPFocus-really-work-
currentMIPFocus = CSProb2.Params.MIPFocus
print("The current MIPFocus is: ",currentMIPFocus)
CSProb2.Params.MIPFocus=2


# Optimize the model
CSProb2.optimize()

Default Time Limit:  inf
Default Interger Tolerance:  1e-05
Set parameter IntFeasTol to value 1e-09
Default Feasibility Tolerance:  1e-06
Set parameter FeasibilityTol to value 1e-09
Default Optimality Tolerance:  1e-06
Set parameter OptimalityTol to value 1e-09
Default MIP Gap Limit:  0.0001
Set parameter MIPGap to value 0
Assigned MIP Gap Limit:  0.0001
The current MIPFocus is:  0
Set parameter MIPFocus to value 2
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (win64)

CPU model: Intel(R) Core(TM) i3-10110U CPU @ 2.10GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 25 rows, 270 columns and 736 nonzeros
Model fingerprint: 0x9b951f27
Variable types: 0 continuous, 270 integer (0 binary)
Coefficient statistics:
  Matrix range     [2e-02, 6e+00]
  Objective range  [3e-01, 3e-01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [8e+00, 1e+03]
Presolve removed 4 rows and 144 columns
Presolve time: 0.

# STANDARDs MINIMIZATION

In [29]:
# STANDARDs MINIMIZATION
M=100000

# Create a new Gurobi model
CSProb3 = gp.Model("Stock Cutting with Unique Standard length usage minimization")

#Debugging Mode
CSProb3.setParam('OutputFlag', 1)

# Variables
x_3 = {}
b_3 = {}

for c in patterns_df:
    x_3[c] = CSProb3.addVar(lb=0, vtype=GRB.INTEGER, name=f'x_3[{c}]')

for sl in unique_elements_row:
    b_3[float(sl)] = CSProb3.addVar(vtype=GRB.BINARY, name=f'b_3[{sl}]')

    
    
    
# Constraints

for sl in unique_elements_row:
    CSProb3.addConstr( gp.quicksum(x_3[c] for c in sl_key__value_c_mapping_dict[sl]) >= b_3[float(sl)])
    CSProb3.addConstr( M*b_3[float(sl)] >= gp.quicksum(x_3[c] for c in sl_key__value_c_mapping_dict[sl]))
    
    

    

# Previous OBJECTIVE 1: Minimize Waste : Now treated as constraint
CSProb3.addConstr(gp.quicksum(x_3[c]*last_two_rows[c]["Wasted Length"] for c in patterns_df) <= ceil(CSProb.objVal *1000)/1000,name="Obj1 as Const.")


for l in all_length_of_cuts_allowed:
    CSProb3.addConstr(gp.quicksum(x_3[c] * new_df[c][l] for c in patterns_df) >= requirements[l],name=f"Requirement const. for {l}")


# Previous Objective 2: Minimize Cost : Now treated as constraint
CSProb3.addConstr(gp.quicksum(x_3[c] * DashBoard_Dataset_StandardLengths["Associated Cost"][float(last_two_rows[c]["Standard Max Length"])] for c in patterns_df) <= ceil(CSProb2.objVal *1000)/1000, name="Obj2 as Const.")


# New Objective 3 to Minimize the number of Unique Standard Length Usage
CSProb3.setObjective(gp.quicksum(b_3[float(sl)] for sl in unique_elements_row),sense=GRB.MINIMIZE)






# Set a time limit for optimization (e.g., 60 seconds)
#https://www.gurobi.com/documentation/10.0/refman/python_parameter_examples.html#PythonParameterExamples
currentTIMElimit = CSProb3.Params.timeLimit
print("Default Time Limit: ",currentTIMElimit)
#CSProb3.Params.timeLimit = Timelimit_for_each_final_CSP_problem_minimizing_WCSP


currentIntFeasTolerancelimit = CSProb3.Params.IntFeasTol
print("Default Interger Tolerance: ",currentIntFeasTolerancelimit)
CSProb3.Params.IntFeasTol = 1e-9


#FeasibilityTol
#https://www.gurobi.com/documentation/current/refman/feasibilitytol.html#parameter:FeasibilityTol
currentFeasibilityTolerancelimit = CSProb3.Params.FeasibilityTol
print("Default Feasibility Tolerance: ",currentFeasibilityTolerancelimit)
CSProb3.Params.FeasibilityTol = 1e-9


#OptimalityTol
#https://www.gurobi.com/documentation/current/refman/optimalitytol.html
currentOptimalityTolerancelimit = CSProb3.Params.OptimalityTol
print("Default Optimality Tolerance: ",currentOptimalityTolerancelimit)
CSProb3.Params.OptimalityTol = 1e-9


currentMIPGapLimit = CSProb3.Params.MIPGap
print("Default MIP Gap Limit: ",currentMIPGapLimit)
#CSProb.setParam('MIPGap', 0)
CSProb3.Params.MIPGap = 0
print("Assigned MIP Gap Limit: ",currentMIPGapLimit)

#https://www.gurobi.com/documentation/current/refman/mip_models.html#:~:text=The%20MIPFocus%20parameter%20allows%20you%20to%20modify%20your,good%20quality%20feasible%20solutions%2C%20you%20can%20select%20MIPFocus%3D1.
#https://www.gurobi.com/documentation/current/refman/mipfocus.html
#https://support.gurobi.com/hc/en-us/community/posts/8791754564369-How-does-MIPFocus-really-work-
currentMIPFocus = CSProb3.Params.MIPFocus
print("The current MIPFocus is: ",currentMIPFocus)
CSProb3.Params.MIPFocus=2

#CSProb3.Params.NumericFocus = 3  # Experiment with different values (0, 1, 2, 3)


# Optimize the model
CSProb3.optimize()

Default Time Limit:  inf
Default Interger Tolerance:  1e-05
Set parameter IntFeasTol to value 1e-09
Default Feasibility Tolerance:  1e-06
Set parameter FeasibilityTol to value 1e-09
Default Optimality Tolerance:  1e-06
Set parameter OptimalityTol to value 1e-09
Default MIP Gap Limit:  0.0001
Set parameter MIPGap to value 0
Assigned MIP Gap Limit:  0.0001
The current MIPFocus is:  0
Set parameter MIPFocus to value 2
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (win64)

CPU model: Intel(R) Core(TM) i3-10110U CPU @ 2.10GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 36 rows, 275 columns and 1556 nonzeros
Model fingerprint: 0xefc27e2b
Variable types: 0 continuous, 275 integer (5 binary)
Coefficient statistics:
  Matrix range     [2e-02, 1e+05]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [8e+00, 1e+03]
Presolve added 5 rows and 0 columns
Presolve removed 0 r

In [30]:
# IMPORTANT

# #Identify Infeasible Constraints:
# #Use the infeasibility information (model.IIS) to identify the constraints causing infeasibility. This can be done as follows:

# CSProb3.computeIIS()
# print('IIS constraints:')
# for constr in CSProb3.getConstrs():
#     if constr.IISConstr:
#         print(constr.ConstrName)

In [31]:
# Print variable values
for var in CSProb3.getVars():
    print(f'{var.varName}: {var.x}')

# # Print constraint values
# for constr in CSProb3.getConstrs():
#     print(f'{constr.ConstrName}: {constr.Pi} (dual value)')

# Print objective value
print(f'Objective Value: {CSProb3.objVal}')

x_3[Combination 1]: 48.0
x_3[Combination 2]: -0.0
x_3[Combination 3]: -0.0
x_3[Combination 4]: -0.0
x_3[Combination 5]: -0.0
x_3[Combination 6]: -0.0
x_3[Combination 7]: -0.0
x_3[Combination 8]: -0.0
x_3[Combination 9]: -0.0
x_3[Combination 10]: -0.0
x_3[Combination 11]: -0.0
x_3[Combination 12]: 24.0
x_3[Combination 13]: -0.0
x_3[Combination 14]: -0.0
x_3[Combination 15]: -0.0
x_3[Combination 16]: -0.0
x_3[Combination 17]: -0.0
x_3[Combination 18]: -0.0
x_3[Combination 19]: -0.0
x_3[Combination 20]: -0.0
x_3[Combination 21]: -0.0
x_3[Combination 22]: -0.0
x_3[Combination 23]: -0.0
x_3[Combination 24]: -0.0
x_3[Combination 25]: -0.0
x_3[Combination 26]: -0.0
x_3[Combination 27]: -0.0
x_3[Combination 28]: -0.0
x_3[Combination 29]: -0.0
x_3[Combination 30]: -0.0
x_3[Combination 31]: -0.0
x_3[Combination 32]: -0.0
x_3[Combination 33]: -0.0
x_3[Combination 34]: -0.0
x_3[Combination 35]: -0.0
x_3[Combination 36]: -0.0
x_3[Combination 37]: -0.0
x_3[Combination 38]: -0.0
x_3[Combination 39]: 

# PATTERNs MINIMIZATION

In [32]:
# Create a new Gurobi model
CSProb4 = gp.Model("Stock Cutting with Minimum Unique Pattern usage")

# Variables
x_4 = {}
y_4 = {}
b_4 = {}

for c in patterns_df:
    x_4[c] = CSProb4.addVar(lb=0, vtype=GRB.INTEGER, name=f'x_4[{c}]')
    y_4[c] = CSProb4.addVar(vtype=GRB.BINARY, name=f'y_4[{c}]')

for sl in unique_elements_row:
    b_4[sl] = CSProb4.addVar(vtype=GRB.BINARY, name=f'b_4[{sl}]')

# Constraints
for c in patterns_df:
    #CSProb4.addConstr(x_4[c] >= b_4[last_two_rows[c]["Standard Max Length"]]) #Wrong
    CSProb4.addConstr(x_4[c] >= y_4[c])
    #CSProb4.addConstr(M * b_4[last_two_rows[c]["Standard Max Length"]] >= x_4[c]) #Wrong
    CSProb4.addConstr(M * y_4[c] >= x_4[c])

    
# Constraints
for sl in unique_elements_row:
    CSProb4.addConstr( gp.quicksum(x_4[c] for c in sl_key__value_c_mapping_dict[sl]) >= b_4[float(sl)])
    CSProb4.addConstr( M*b_4[float(sl)] >= gp.quicksum(x_4[c] for c in sl_key__value_c_mapping_dict[sl]))
    
    
# Previous OBJECTIVE 1: Minimize Waste : Now treated as constraint
CSProb4.addConstr(gp.quicksum(x_4[c]*last_two_rows[c]["Wasted Length"] for c in patterns_df) <= ceil(CSProb.objVal *1000)/1000)

# Satisfying Requirements
for l in all_length_of_cuts_allowed:
    CSProb4.addConstr(gp.quicksum(x_4[c] * new_df[c][l] for c in patterns_df) >= requirements[l])

# Previous Objective 2: Minimize Cost : Now treated as constraint
CSProb4.addConstr(gp.quicksum(x_4[c] * DashBoard_Dataset_StandardLengths["Associated Cost"][float(last_two_rows[c]["Standard Max Length"])] for c in patterns_df) <= ceil(CSProb2.objVal *1000)/1000)




# Previous Objective 3: to Minimize the number of Unique Standard Length Usage
CSProb4.addConstr(gp.quicksum(b_4[sl] for sl in unique_elements_row) <= ceil(CSProb3.objVal *1000)/1000)


# New Objective 4 to Minimize the number of Unique Pattern Usage
CSProb4.setObjective(gp.quicksum(y_4[c] for c in patterns_df),sense=GRB.MINIMIZE)





# Set a time limit for optimization (e.g., 60 seconds)
#https://www.gurobi.com/documentation/10.0/refman/python_parameter_examples.html#PythonParameterExamples
currentTIMElimit = CSProb4.Params.timeLimit
print("Default Time Limit: ",currentTIMElimit)
#CSProb4.Params.timeLimit = Timelimit_for_each_final_CSP_problem_minimizing_WCSP


currentIntFeasTolerancelimit = CSProb4.Params.IntFeasTol
print("Default Interger Tolerance: ",currentIntFeasTolerancelimit)
CSProb4.Params.IntFeasTol = 1e-9


#FeasibilityTol
#https://www.gurobi.com/documentation/current/refman/feasibilitytol.html#parameter:FeasibilityTol
currentFeasibilityTolerancelimit = CSProb4.Params.FeasibilityTol
print("Default Feasibility Tolerance: ",currentFeasibilityTolerancelimit)
CSProb4.Params.FeasibilityTol = 1e-9


#OptimalityTol
#https://www.gurobi.com/documentation/current/refman/optimalitytol.html
currentOptimalityTolerancelimit = CSProb4.Params.OptimalityTol
print("Default Optimality Tolerance: ",currentOptimalityTolerancelimit)
CSProb4.Params.OptimalityTol = 1e-9


currentMIPGapLimit = CSProb4.Params.MIPGap
print("Default MIP Gap Limit: ",currentMIPGapLimit)
#CSProb.setParam('MIPGap', 0)
CSProb4.Params.MIPGap = 0
print("Default MIP Gap Limit: ",currentMIPGapLimit)

#https://www.gurobi.com/documentation/current/refman/mip_models.html#:~:text=The%20MIPFocus%20parameter%20allows%20you%20to%20modify%20your,good%20quality%20feasible%20solutions%2C%20you%20can%20select%20MIPFocus%3D1.
#https://www.gurobi.com/documentation/current/refman/mipfocus.html
#https://support.gurobi.com/hc/en-us/community/posts/8791754564369-How-does-MIPFocus-really-work-
currentMIPFocus = CSProb4.Params.MIPFocus
print("The current MIPFocus is: ",currentMIPFocus)
CSProb4.Params.MIPFocus=2


# Optimize the model
CSProb4.optimize()

Default Time Limit:  inf
Default Interger Tolerance:  1e-05
Set parameter IntFeasTol to value 1e-09
Default Feasibility Tolerance:  1e-06
Set parameter FeasibilityTol to value 1e-09
Default Optimality Tolerance:  1e-06
Set parameter OptimalityTol to value 1e-09
Default MIP Gap Limit:  0.0001
Set parameter MIPGap to value 0
Default MIP Gap Limit:  0.0001
The current MIPFocus is:  0
Set parameter MIPFocus to value 2
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (win64)

CPU model: Intel(R) Core(TM) i3-10110U CPU @ 2.10GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 577 rows, 545 columns and 2641 nonzeros
Model fingerprint: 0x169dd4fb
Variable types: 0 continuous, 545 integer (275 binary)
Coefficient statistics:
  Matrix range     [2e-02, 1e+05]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [4e+00, 1e+03]
Presolve added 5 rows and 5 columns
Presolve time: 0.0

# Populate Result Excel Sheet

In [33]:
end_time_is_this=time.time()
print(end_time_is_this)

actual_time_taken = end_time_is_this - start_time_is_this
actual_time_taken

1702800984.3424425


61.8032751083374

In [34]:
winsound.Beep(500,1000)
wb_final=openpyxl.Workbook()

sheet_for_Usage = wb_final.active
sheet_for_Usage.title = 'Variable x and y'
sheet_for_b = wb_final.create_sheet(title='Variable b')
sheet_for_Objectives = wb_final.create_sheet(title='Objectives')

#sheet_for_x
row_number=1
sheet_for_Usage.cell(row=row_number,column=1).value= "PATTERN"
sheet_for_Usage.cell(row=row_number,column=2).value= "Standard Length"
sheet_for_Usage.cell(row=row_number,column=3).value= "No. of Usage in Objective 1 (x)"
sheet_for_Usage.cell(row=row_number,column=4).value= "No. of Usage in Objective 2 (x_2)"
sheet_for_Usage.cell(row=row_number,column=5).value= "No. of Usage in Objective 3 (x_3)"
sheet_for_Usage.cell(row=row_number,column=6).value= "No. of Usage in Objective 4 (x_4)"
sheet_for_Usage.cell(row=row_number,column=7).value= "Unique Pattern Usages in Objective 4 (y_4)"

for c in patterns_df:
    row_number+=1
    sheet_for_Usage.cell(row=row_number,column=1).value=c
    sheet_for_Usage.cell(row=row_number,column=2).value=last_two_rows[c]["Standard Max Length"]
    sheet_for_Usage.cell(row=row_number,column=3).value=x[c].x
    sheet_for_Usage.cell(row=row_number,column=4).value=x_2[c].x
    sheet_for_Usage.cell(row=row_number,column=5).value=x_3[c].x
    sheet_for_Usage.cell(row=row_number,column=6).value=x_4[c].x
    sheet_for_Usage.cell(row=row_number,column=7).value=y_4[c].x
    #if x_3[c].x:
    #    print(c, " used " , x_3[c].x , "times, with standard length", last_two_rows[c]["Standard Max Length"])


    
#sheet_for_b
row_number=1
sheet_for_b.cell(row=row_number,column=1).value= "Standard Length"
sheet_for_b.cell(row=row_number,column=2).value= "Utilization in Objective 3 (b-3)"
sheet_for_b.cell(row=row_number,column=3).value= "Utilization in Objective 4 (b_4)"

for sl in unique_elements_row:
    row_number+=1
    sheet_for_b.cell(row=row_number,column=1).value=sl
    sheet_for_b.cell(row=row_number,column=2).value=b_3[sl].x
    sheet_for_b.cell(row=row_number,column=3).value=b_4[sl].x
    
    
#sheet_for_Objectives
row_number=1
sheet_for_Objectives.cell(row=row_number,column=1).value= "Objective 1"
sheet_for_Objectives.cell(row=row_number,column=2).value= "Objective 2"
sheet_for_Objectives.cell(row=row_number,column=3).value= "Objective 3"
sheet_for_Objectives.cell(row=row_number,column=4).value= "Objective 4"
sheet_for_Objectives.cell(row=row_number,column=5).value= "Total Time taken for the Pattern Generations and all Objective Computations"
row_number=2
sheet_for_Objectives.cell(row=row_number,column=1).value= "Minimum Total WASTE"
sheet_for_Objectives.cell(row=row_number,column=2).value= "Minimum Total COST"
sheet_for_Objectives.cell(row=row_number,column=3).value= "Minimum Unique STANDARD LENGTH Usage"
sheet_for_Objectives.cell(row=row_number,column=4).value= "Minimum Unique PATTERN Usage"
sheet_for_Objectives.cell(row=row_number,column=5).value= "(Seconds)"
row_number=3
sheet_for_Objectives.cell(row=row_number,column=1).value= CSProb.objVal
sheet_for_Objectives.cell(row=row_number,column=2).value= CSProb2.objVal
sheet_for_Objectives.cell(row=row_number,column=3).value= CSProb3.objVal
sheet_for_Objectives.cell(row=row_number,column=4).value= CSProb4.objVal
sheet_for_Objectives.cell(row=row_number,column=5).value= actual_time_taken


Final_output_file_path = "./Prob"+str(Decisions["Problem"])+"_PatternUsage_with_WCSP_Minimization"+".xlsx"
wb_final.save(Final_output_file_path)

In [35]:
# winsound.Beep(500,1000)
# wb_final=openpyxl.Workbook()

# sheet_for_Usage = wb_final.active

# row_number=1
# sheet_for_Usage.cell(row=row_number,column=1).value= "PATTERN"
# sheet_for_Usage.cell(row=row_number,column=2).value= "Standard Length"
# sheet_for_Usage.cell(row=row_number,column=3).value= "No. of Usage"


# for c in patterns_df:
#     row_number+=1
#     sheet_for_Usage.cell(row=row_number,column=1).value=c
#     sheet_for_Usage.cell(row=row_number,column=2).value=last_two_rows[c]["Standard Max Length"]
#     sheet_for_Usage.cell(row=row_number,column=3).value=x[c].x
#     if x[c].x:
#         print(c, " used " , x[c].x , "times, with standard length", last_two_rows[c]["Standard Max Length"])

        
# row_number+=3
# sheet_for_Usage.cell(row=row_number,column=1).value= "Minimum Trim Loss"
# sheet_for_Usage.cell(row=row_number,column=2).value= "Objective Value:"
# sheet_for_Usage.cell(row=row_number,column=3).value= CSProb.objVal
# print("Objective Value:", CSProb.objVal)

# Final_output_file_path = "./Problem_"+str(Decisions["Problem"])+"__PatternUsage_withMinimumTrimLoss"+"_Gurobi.xlsx"
# wb_final.save(Final_output_file_path)

In [36]:
# winsound.Beep(500,1000)
# wb_final=openpyxl.Workbook()

# sheet_for_Usage = wb_final.active

# row_number=1
# sheet_for_Usage.cell(row=row_number,column=1).value= "PATTERN"
# sheet_for_Usage.cell(row=row_number,column=2).value= "Standard Length"
# sheet_for_Usage.cell(row=row_number,column=3).value= "No. of Usage"


# for c in patterns_df:
#     row_number+=1
#     sheet_for_Usage.cell(row=row_number,column=1).value=c
#     sheet_for_Usage.cell(row=row_number,column=2).value=last_two_rows[c]["Standard Max Length"]
#     sheet_for_Usage.cell(row=row_number,column=3).value=x_2[c].x
#     if x_2[c].x:
#         print(c, " used " , x_2[c].x , "times, with standard length", last_two_rows[c]["Standard Max Length"])
        
# row_number+=3
# sheet_for_Usage.cell(row=row_number,column=1).value= "Minimum Total Cost"
# sheet_for_Usage.cell(row=row_number,column=2).value= "Objective Value:"
# sheet_for_Usage.cell(row=row_number,column=3).value= CSProb2.objVal
# print("Objective Value:", CSProb2.objVal)

# Final_output_file_path = "./Problem_"+str(Decisions["Problem"])+"__PatternUsage_withMinimumCost"+"_Gurobi.xlsx"
# wb_final.save(Final_output_file_path)